# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import os
import glob
import csv
import pandas as pd 

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
#print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list=[]
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open (f, 'r', encoding='utf8') as csvfile:
        # creating a csv reader object
        csvreader=csv.reader(csvfile)
        next(csvreader) # skip the header
        # extracting each data row one by one and append it to full_data_rows_list
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

In [4]:
print(len(full_data_rows_list))

8056


#### Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables

In [5]:
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == '' or len (row)==1):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
df=pd.read_csv('event_datafile.csv')

In [7]:
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [8]:
df.shape

(6809, 11)

# Part II. Modeling data with Apache Cassandra . 

#### Now after we pre-processed the data and filled in <font color=red>event_datafile.csv</font> which is located within the Workspace directory.  The event_datafile.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId



#### Creating a Cluster

In [9]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace ' omar_udacity'

In [10]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS omar_udacity
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor':1}
                    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [11]:
try:
    session.set_keyspace("omar_udacity")
except Exception as e:
    print (e)

### Create queries (data model) to ask the following three questions of the data:

#### Query 1: the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

To run a query that gives the artist name, song title and song's length based on the sessionId & itemInSession, 
I model the data in a way that it responds to the specific question below :Since my data is looking for 
sessionId & itemInSession the PRIMARY KEY would be composed of these 2 attributes.

In [12]:
query="CREATE TABLE IF NOT EXISTS songs_details_by_session_details"
query= query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY ( sessionId, itemInSession))"
try :
    session.execute(query)
except Exception as e :
    print(e)

Open the `event_datafile.csv` file, process it to insert the specific data into `songs_details_by_session_details` table.

The int(line[ ]) and float(line[ ]) are used to avoid errors when the datatable is excepting something and receives other datatypes. 

In [13]:
file = 'event_datafile.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_details_by_session_details (sessionId, itemInSession,artist, song, length)"
        query = query + "VALUES ( %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line [3]), line[0],line[9],float(line[5])))

##### Quality check for data model 1

In [14]:
# to verify that the data have been inserted into each table, I run the following SELECT statement
query= " SELECT artist, song, length from songs_details_by_session_details WHERE sessionId=338 AND itemInSession=4"
try:
    rows=session.execute(query)
except Exception as e :
    print(e)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To run a query that gives the artist name, song title (sorted by itemInSession) 
and user first & last based userid = 10 & sessionid = 182. 
I model the data in a way that it responds to the specific question above :Since my data is looking for 
userid = 10 & sessionid = 182. 
+ the PRIMARY KEY is composed of these 2 attributes (the partition key (userId & sessionId) and clustering key).
+ the itemInSession column is the clustering key to have song title sorted by itemInSession.

In [15]:
query="CREATE TABLE IF NOT EXISTS song_details_by_userANDsession_id"
query=query+"""(userId int, sessionId int, itemInSession int, \
                firstName text, lastName text,  artist text, song text, \
                PRIMARY KEY ((userId, sessionId), itemInSession)
                )"""
try:
    session.execute(query)
except Exception as e :
    print(e)

Open the `event_datafile.csv` file, process it to insert the specific data into `song_details_by_userANDsession_id` table.

The int(line[ ])is used to avoid errors when the datatable is excepting something and receives other datatypes 

In [16]:

file = 'event_datafile.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_details_by_userANDsession_id (userId, sessionId ,itemInSession ,firstname, lastname, artist, song)"
        query = query + "VALUES ( %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line [8]), int(line[3]),line[1],line[4], line[0], line[9]))

In [17]:
# to verify that the data have been inserted into each table, I run the following SELECT statement
query= " SELECT firstname, lastname, artist, song from song_details_by_userANDsession_id WHERE  userId=10 AND sessionId=182  "
try:
    rows=session.execute(query)
except Exception as e :
    print(e)
for row in rows:
    print(row.firstname, row.lastname, row.artist, row.song)

Sylvie Cruz Down To The Bone Keep On Keepin' On
Sylvie Cruz Three Drives Greece 2000
Sylvie Cruz Sebastien Tellier Kilometer
Sylvie Cruz Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


#### Query 3: every user name (first and last) in my music app history who listened to the song = 'All Hands Against His Own' ,  

since my data is looking for song = 'All Hands Against His Own', the PRIMARY KEY would be : 
+ the song name as the partition key  
+ the userId column as the clustering key (for sorting) 

In [18]:
query="CREATE TABLE IF NOT EXISTS user_name_listening_to_specific_songs \
        (song text,userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
file='event_datafile.csv'
with open (file, encoding='utf8') as f :
    csvreader=csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query="INSERT INTO user_name_listening_to_specific_songs (song, userId, firstName, lastName)"
        query=query+"VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]),line[1],line[4]))

In [20]:
# to verify that the data have been inserted into each table, I run the following SELECT statement
query2= " SELECT firstname, lastname from user_name_listening_to_specific_songs WHERE song = 'All Hands Against His Own'"
try:
    rows=session.execute(query2)
except Exception as e :
    print(e)
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
try :
    session.execute("DROP TABLE songs_details_by_session_details ")
except Exception as e :
    print(e)

In [22]:
try :
    session.execute("DROP TABLE song_details_by_userANDsession_id ")
except Exception as e :
    print(e)

In [23]:
try :
    session.execute("DROP TABLE user_name_listening_to_specific_songs")
except Exception as e :
    print(e)

### Close the session and cluster connection

In [24]:
session.shutdown()
cluster.shutdown()